# Setup & Load Clean Data

In [1]:
# ============================================================
# Setup & Environment for Feature Engineering
# Generates 5 datasets:
# structured, text, image, regression, classification
# ============================================================

# --- Core Imports ---
import os
import pandas as pd
import numpy as np
from pathlib import Path
from io import BytesIO
from PIL import Image, UnidentifiedImageError
from tqdm import tqdm
import requests

# --- ML / Image Processing ---
import torch
import torch.nn as nn
%pip install torchvision --quiet
import torchvision.transforms as transforms
from torchvision import models

# --- Text & Sentiment ---
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD

# --- Paths ---
base = Path.cwd().parent
processed_path = base / "data" / "processed"

# --- Load Cleaned Dataset ---
df = pd.read_parquet(processed_path / "youtube_clean_final.parquet")
print("✅ Loaded cleaned dataset:" , df.shape)

# --- Device & ResNet50 Model Setup (for Thumbnail Features) ---
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Load pretrained ResNet50 (ImageNet weights)
resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
resnet50.fc = nn.Identity()  # remove final classifier
resnet50 = resnet50.to(device)
resnet50.eval()

# Define preprocessing transform (ImageNet normalization)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
print("✅ ResNet50 loaded and ready for image feature extraction.")
df.head()

Note: you may need to restart the kernel to use updated packages.
✅ Loaded cleaned dataset: (5742, 19)
✅ ResNet50 loaded and ready for image feature extraction.


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,subscribers,views_per_subscriber,views_per_subscriber_log
0,-0CMnp02rNY,18.11.06,Mindy Kaling's Daughter Had the Perfect Reacti...,TheEllenShow,24,2018-06-04 13:00:00+00:00,"ellen|""ellen degeneres""|""the ellen show""|""elle...",800359,9773,332,423,https://i.ytimg.com/vi/-0CMnp02rNY/default.jpg,False,False,False,Ocean's 8 star Mindy Kaling dished on bringing...,2.376002e+07,0.033685,0.033130
1,-0NYY8cqdiQ,18.01.02,Megan Mullally Didn't Notice the Interesting P...,TheEllenShow,24,2018-01-29 14:00:39+00:00,"megan mullally|""megan""|""mullally""|""will and gr...",563746,4429,54,94,https://i.ytimg.com/vi/-0NYY8cqdiQ/default.jpg,False,False,False,Ellen and Megan Mullally have known each other...,2.376002e+07,0.023727,0.023450
2,-1Hm41N0dUs,18.01.05,Cast of Avengers: Infinity War Draws Their Cha...,Jimmy Kimmel Live,23,2018-04-27 07:30:02+00:00,"jimmy|""jimmy kimmel""|""jimmy kimmel live""|""late...",2058516,41248,580,1484,https://i.ytimg.com/vi/-1Hm41N0dUs/default.jpg,False,False,False,"Benedict Cumberbatch, Don Cheadle, Elizabeth O...",1.126290e+07,0.182770,0.167859
3,-1yT-K3c6YI,17.02.12,YOUTUBER QUIZ + TRUTH OR DARE W/ THE MERRELL T...,Molly Burke,22,2017-11-28 18:30:43+00:00,"youtube quiz|""youtuber quiz""|""truth or dare""|""...",231341,7734,212,846,https://i.ytimg.com/vi/-1yT-K3c6YI/default.jpg,False,False,False,Check out the video we did on the Merrell Twin...,2.740040e+05,0.844295,0.612097
4,-2RVw2_QyxQ,17.16.11,2017 Champions Showdown: Day 3,Saint Louis Chess Club,27,2017-11-12 02:39:01+00:00,"Chess|""Saint Louis""|""Club""",71089,460,27,20,https://i.ytimg.com/vi/-2RVw2_QyxQ/default.jpg,False,False,False,The Saint Louis Chess Club hosts a series of f...,1.477180e+05,0.481245,0.392883


# Basic Text Cleaning

In [2]:
# --- Clean title column ---
df["title"] = df["title"].astype(str).str.strip()
print("Sample titles:")
print(df["title"].head(5))

Sample titles:
0    Mindy Kaling's Daughter Had the Perfect Reacti...
1    Megan Mullally Didn't Notice the Interesting P...
2    Cast of Avengers: Infinity War Draws Their Cha...
3    YOUTUBER QUIZ + TRUTH OR DARE W/ THE MERRELL T...
4                       2017 Champions Showdown: Day 3
Name: title, dtype: object


# Structured Title Features

In [3]:
# --- Handcrafted title features ---
df["title_length"] = df["title"].apply(len)
df["word_count"] = df["title"].apply(lambda x: len(x.split()))
df["caps_ratio"] = df["title"].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)
df["has_question"] = df["title"].apply(lambda x: int("?" in x))
df["has_exclamation"] = df["title"].apply(lambda x: int("!" in x))
df["has_number"] = df["title"].apply(lambda x: int(any(ch.isdigit() for ch in x)))
df["avg_word_len"] = df["title"].apply(lambda x: np.mean([len(w) for w in x.split()]) if len(x.split()) > 0 else 0)

print("✅ Added basic title features.")
df[["title", "title_length", "word_count", "caps_ratio", "has_question", "has_exclamation", "has_number", "avg_word_len"]].head()

✅ Added basic title features.


,title,title_length,word_count,caps_ratio,has_question,has_exclamation,has_number,avg_word_len
0,Mindy Kaling's Daughter Had the Perfect Reacti...,74,11,0.121622,0,0,0,5.818182
1,Megan Mullally Didn't Notice the Interesting P...,75,10,0.106667,0,0,0,6.600000
2,Cast of Avengers: Infinity War Draws Their Cha...,53,8,0.132075,0,0,0,5.750000
3,YOUTUBER QUIZ + TRUTH OR DARE W/ THE MERRELL T...,51,10,0.764706,0,1,0,4.200000
4,2017 Champions Showdown: Day 3,30,5,0.100000,0,0,1,5.200000


# Sentiment Feature

In [4]:
# --- Sentiment analysis ---
analyzer = SentimentIntensityAnalyzer()

df["sentiment_vader"] = df["title"].apply(lambda x: analyzer.polarity_scores(x)["compound"])

print("✅ Added sentiment feature.")
df[["title", "sentiment_vader"]].head()

✅ Added sentiment feature.


,title,sentiment_vader
0,Mindy Kaling's Daughter Had the Perfect Reacti...,0.5719
1,Megan Mullally Didn't Notice the Interesting P...,-0.3089
2,Cast of Avengers: Infinity War Draws Their Cha...,-0.5994
3,YOUTUBER QUIZ + TRUTH OR DARE W/ THE MERRELL T...,0.5147
4,2017 Champions Showdown: Day 3,0.5267


# Structured Feature Matrix + Targets

In [5]:
# --- Structured features ---
structured_features = [
    "title_length", "word_count", "caps_ratio",
    "has_question", "has_exclamation", "has_number", "avg_word_len",
    "sentiment_vader",
    "subscribers"
]

X_structured = df[structured_features].copy()

# --- Targets ---
y_reg = df["views_per_subscriber"]       # regression target (continuous)
y_clf = (df["views_per_subscriber"] >= df["views_per_subscriber"].quantile(0.75)).astype(int)  # top 25% success

print("Structured feature matrix:", X_structured.shape)
print("Regression target shape:", y_reg.shape)
print("Classification target distribution:\n", y_clf.value_counts(normalize=True))

Structured feature matrix: (5742, 9)
Regression target shape: (5742,)
Classification target distribution:
 views_per_subscriber
0    0.749913
1    0.250087
Name: proportion, dtype: float64


In [6]:
# --- Handle outliers for regression target ---

# Option 1: Clip extreme values (simple and intuitive)
df["views_per_subscriber"] = np.clip(df["views_per_subscriber"], 0, 500)

# Option 2 (alternative): Use log transform for smoother distribution
# df["views_per_subscriber_log"] = np.log1p(df["views_per_subscriber"])

# Then redefine regression target variable to use this cleaned version
y_reg = df["views_per_subscriber"]

# (If you used log version, change above line to y_reg = df["views_per_subscriber_log"])

# Save Structured Datasets

In [7]:
# --- Save structured features and targets ---
X_structured.to_parquet(processed_path / "youtube_features_structured.parquet", index=False)
y_reg.to_frame("views_per_subscriber").to_parquet(processed_path / "youtube_target_regression.parquet", index=False)
y_clf.to_frame("high_clickability").to_parquet(processed_path / "youtube_target_classification.parquet", index=False)

print("✅ Saved structured features and targets.")
print("✅ Saved cleaned regression target to:", processed_path / "youtube_target_regression.parquet")
print("✅ Saved cleaned classification target to:", processed_path / "youtube_target_classification.parquet")

✅ Saved structured features and targets.
✅ Saved cleaned regression target to: c:\Users\bhada\Documents\GitHub\YouTube_Clickability_Study\data\processed\youtube_target_regression.parquet
✅ Saved cleaned classification target to: c:\Users\bhada\Documents\GitHub\YouTube_Clickability_Study\data\processed\youtube_target_classification.parquet


# TF-IDF Text Features (Unigrams + Bigrams)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# --- TF-IDF setup ---
tfidf = TfidfVectorizer(
    max_features=1000,
    stop_words="english",
    ngram_range=(1, 2)
)

tfidf_matrix = tfidf.fit_transform(df["title"])
print("Raw TF-IDF shape:", tfidf_matrix.shape)

# --- Dimensionality reduction (TruncatedSVD) ---
svd = TruncatedSVD(n_components=50, random_state=42)
tfidf_reduced = svd.fit_transform(tfidf_matrix)

print("Reduced TF-IDF shape:", tfidf_reduced.shape)

# --- Convert to DataFrame ---
tfidf_cols = [f"tfidf_comp_{i+1}" for i in range(tfidf_reduced.shape[1])]
X_tfidf = pd.DataFrame(tfidf_reduced, columns=tfidf_cols)

X_tfidf.head()

Raw TF-IDF shape: (5742, 1000)
Reduced TF-IDF shape: (5742, 50)


,tfidf_comp_1,tfidf_comp_2,tfidf_comp_3,tfidf_comp_4,tfidf_comp_5,tfidf_comp_6,tfidf_comp_7,tfidf_comp_8,tfidf_comp_9,tfidf_comp_10,...,tfidf_comp_41,tfidf_comp_42,tfidf_comp_43,tfidf_comp_44,tfidf_comp_45,tfidf_comp_46,tfidf_comp_47,tfidf_comp_48,tfidf_comp_49,tfidf_comp_50
0,0.008023,0.000299,0.021451,0.021926,0.006446,0.001927,0.006493,0.000916,0.020432,0.009336,...,0.152593,0.105763,-0.016033,0.039567,-0.121613,0.124725,-0.055770,0.075234,-0.116634,-0.063000
1,0.000881,0.000389,0.009124,0.016396,0.013081,0.001009,-0.002582,-0.009785,0.002011,0.005803,...,0.005325,-0.001931,-0.011767,-0.023423,-0.006530,0.000859,-0.010738,-0.006718,-0.012869,0.005131
2,0.010647,-0.010526,0.007540,0.021256,-0.014330,-0.005137,-0.030994,-0.000343,0.002833,-0.002526,...,0.263607,-0.058746,0.033722,-0.141856,0.011770,0.208311,-0.078890,-0.121032,0.063431,-0.285299
3,0.003308,-0.002960,0.005910,0.005595,-0.000852,0.000878,0.001912,0.002622,0.004437,0.008354,...,-0.013322,0.002138,-0.000899,-0.000225,0.000905,0.011466,0.006203,0.005829,0.006671,-0.001436
4,0.020000,-0.014160,0.172012,0.249537,0.296775,-0.027703,-0.003996,-0.180619,0.016951,0.076816,...,0.005867,-0.010501,0.023246,0.011764,-0.012430,0.013665,0.033865,0.024061,0.039195,-0.036545


In [9]:
# see what each component means
terms = tfidf.get_feature_names_out()
for i, comp in enumerate(svd.components_[:5]):  # first 5 components
    top_terms = [terms[x] for x in comp.argsort()[-10:][::-1]]
    print(f"Component {i+1}: {', '.join(top_terms)}")


Component 1: official, trailer, video, official trailer, official video, hd, trailer hd, netflix, hd netflix, music
Component 2: video, official video, music, music video, official, official music, ft, lyric, lyric video, video ft
Component 3: 2018, 2017, new, vs, live, game, day, best, awards, 10
Component 4: 2017, new, star, wars, star wars, vs, live, makeup, jedi, best
Component 5: 2017, vs, best, 10, highlights, awards, nfl, game, live, christmas


# Save TF-IDF Dataset

In [10]:
# --- Save reduced TF-IDF features ---
X_tfidf.to_parquet(processed_path / "youtube_features_text.parquet", index=False)

print("✅ Saved text-based TF-IDF features to:", processed_path / "youtube_features_text.parquet")

✅ Saved text-based TF-IDF features to: c:\Users\bhada\Documents\GitHub\YouTube_Clickability_Study\data\processed\youtube_features_text.parquet


# Extract Image Features

In [11]:
# --- Downloads thumbnail and returns a 2048-D ResNet50 embedding. ---

def extract_features_from_url(url: str) -> np.ndarray:
    try:
        if not isinstance(url, str) or not url.startswith("http"):
            raise ValueError("Invalid URL")
        # Try high-quality thumbnail if available
        url_hq = url.replace("/default.jpg", "/hqdefault.jpg")
        # Download image
        response = requests.get(url_hq, timeout=8)
        response.raise_for_status()
        # Decode image
        img = Image.open(BytesIO(response.content)).convert("RGB")
        img_tensor = transform(img).unsqueeze(0).to(device)
        # Extract ResNet embedding
        with torch.no_grad():
            features = resnet50(img_tensor).cpu().numpy().flatten()
        return features.astype(np.float32)

    except (requests.exceptions.RequestException, UnidentifiedImageError, ValueError):
        # Return zero vector if the image is missing or invalid
        return np.zeros(2048, dtype=np.float32)

In [12]:
# --- Applies extraction with a progress bar ---

tqdm.pandas(desc="Extracting ResNet features")
df["resnet_features"] = df["thumbnail_link"].progress_apply(extract_features_from_url)

# Count failed downloads
fail_count = sum(np.all(f == 0) for f in df["resnet_features"])
print(f"⚠️ Failed to download or decode {fail_count:,} thumbnails "
      f"({100*fail_count/len(df):.1f}%)")

Extracting ResNet features: 100%|██████████| 5742/5742 [40:37<00:00,  2.36it/s]  


⚠️ Failed to download or decode 534 thumbnails (9.3%)


# Apply PCA (dimensionality reduction)

In [13]:
# Stacks all 2048-D feature vectors into one big matrix.
# Uses PCA to reduce them to 50 features while keeping ≈ 95 % of the important variance.

# --- Remove old PCA columns ---
pca_cols = [c for c in df.columns if c.startswith("pca_")]
if pca_cols:
    df = df.drop(columns=pca_cols)

# --- PCA Computation ---
X = np.vstack(df["resnet_features"].values)
print("Raw feature matrix shape:", X.shape)

pca = PCA(n_components=50, random_state=42)
X_pca = pca.fit_transform(X)

df_pca = pd.DataFrame(X_pca, columns=[f"pca_{i+1}" for i in range(50)])
df = pd.concat([df.reset_index(drop=True), df_pca], axis=1)

print("✅ PCA recomputed → shape:", df_pca.shape)

Raw feature matrix shape: (5742, 2048)
✅ PCA recomputed → shape: (5742, 50)


# Save Image Features Dataset

In [14]:
# Saving the dataset as "youtube_features_image.parquet"

# Keep only the PCA columns
pca_cols = [f"pca_{i}" for i in range(1, 51)]
df_pure_image = df[pca_cols].copy()
df_pure_image.to_parquet(processed_path / "youtube_features_image.parquet", index=False)

print("✅ Saved pure thumbnail dataset to:", processed_path / "youtube_features_image.parquet")
print("Shape:", df_pure_image.shape)

✅ Saved pure thumbnail dataset to: c:\Users\bhada\Documents\GitHub\YouTube_Clickability_Study\data\processed\youtube_features_image.parquet
Shape: (5742, 50)


In [15]:
# Verify saved image feature dataset

df_check = pd.read_parquet(processed_path / "youtube_features_image.parquet")
print(f"✅ Loaded {len(df_check):,} rows × {len(df_check.columns)} columns")
print("Sample columns:", df_check.columns[-10:].to_list())

✅ Loaded 5,742 rows × 50 columns
Sample columns: ['pca_41', 'pca_42', 'pca_43', 'pca_44', 'pca_45', 'pca_46', 'pca_47', 'pca_48', 'pca_49', 'pca_50']


# Summary Check

In [16]:
# Summary of all saved feature datasets
print("✅ Summary of saved datasets:")

print(f"• Structured features:       {X_structured.shape}")
print(f"• Text (TF-IDF) features:    {X_tfidf.shape}")
print(f"• Image (PCA) features:      {df_pure_image.shape}")
print(f"• Regression target:         {y_reg.shape}")
print(f"• Classification target:     {y_clf.shape}")

print("\n📂 All datasets saved under:")
print(processed_path.resolve())

✅ Summary of saved datasets:
• Structured features:       (5742, 9)
• Text (TF-IDF) features:    (5742, 50)
• Image (PCA) features:      (5742, 50)
• Regression target:         (5742,)
• Classification target:     (5742,)

📂 All datasets saved under:
C:\Users\bhada\Documents\GitHub\YouTube_Clickability_Study\data\processed
